In [1]:
import os

In [2]:
pwd

'/Users/vaishalikant/Downloads/data science project/Chest-Cancer-Detection-Mlflow/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/vaishalikant/Downloads/data science project/Chest-Cancer-Detection-Mlflow'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/vishikant/Chest-Cancer-Detection-Mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="vishikant"
os.environ["MLFLOW_TRACKING_PASSWORD"]="13vVV13&"

In [6]:
import tensorflow as tf
model = "/Users/vaishalikant/Downloads/data science project/Chest-Cancer-Detection-Mlflow/artifacts/training/model.h5"


In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/vishikant/Chest-Cancer-Detection-Mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [10]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [11]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [15]:
try:
    # Update CONFIG_FILE_PATH to the correct path if necessary
    CONFIG_FILE_PATH = Path("/Users/vaishalikant/Downloads/data science project/Chest-Cancer-Detection-Mlflow/config/config.yaml")  # Replace with the actual path
    #PARAMS_FILE_PATH = Path("/absolute/path/to/params.yaml")  # Replace with the actual path if needed

    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow() 

except Exception as e:
   raise e

[2025-04-02 18:17:20,848: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-04-02 18:17:20,851: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-02 18:17:20,851: INFO: common: created directory at: artifacts]
[2025-04-02 18:17:20,934: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 102 images belonging to 2 classes.


AttributeError: 'str' object has no attribute 'evaluate'

In [13]:
from cnnClassifier.entity.config_entity import EvaluationConfig